# Example: Writing a power grid model to the extended 3DcityDB

This example shows how to write a power grid model to the extended *3DCityDB*, using the `dblayer` package. Basic knowledge about [CityGML](http://www.citygmlwiki.org) and the [Utility Network ADE](https://en.wiki.utilitynetworks.sig3d.org) are required for this notebook. A nice overview of the of the basic concepts of the Utility Network ADE can be found [here](https://en.wiki.utilitynetworks.sig3d.org/images/upload/20160913_-_Kolbe_-_Modeling_3D_Utility_Networks_%2B_UtilityNetwork_ADE_Core_Model.pdf).

For the example, a [CIGRE network](http://www.cigre.org/News/Network-of-the-Future) is used, as shown in the following figure.
<img src="./figures/cigre_network_lv.png" style="width: 650px;">

## Getting started ...

Import the required funtionality from package `dblayer`.

In [1]:
# Basic functionality.
from dblayer import *

# SQL functions for inserting Utility Network ADE and Energy ADE objects into the database.
from dblayer.func.func_citydb_view_utn import *
from dblayer.func.func_citydb_view_nrg import *

# Helper functions for the Utility Network ADE and generic attributes.
from dblayer.helpers.utn import *
from dblayer.helpers.generic_attributes import *

Define a data structure (using Python's `namedtuple`) for collecting information about bus data that will be stored in the 3DCityDB:
 * ID of the network feature associated to the bus
 * ID of the network feature graph associated to the bus
 * ID of the single node contained in the network feature graph
 * 2D coordinates of the bus

In [2]:
from collections import namedtuple
BusData = namedtuple('BusData', # name of the tuple
                     ['feature_id', 'feature_graph_id', 'node_id', 'posx', 'posy'] ) # variable names of the tuple

Load data from CSV files, containing information about busses, lines and loads.

In [3]:
import pandas as pd
import os.path

busses_csv_data = pd.read_csv(os.path.join('csv', 'bus_data.csv'), index_col=0)
lines_csv_data = pd.read_csv(os.path.join('csv', 'line_data.csv'), index_col=0)
loads_csv_data = pd.read_csv(os.path.join('csv', 'load_data.csv'), index_col = 0)

## Definition of helper functions for adding network components

Defining a network graph requires the definition of a lot of individual components. This can become a very repetitive and cumbersome task, unless one uses a programatic approach. To this end, a bunch of helper functions for adding specific network components are defined in the following, relying on functionality provided by package `dblayer`.

### A function for adding busses

Define a helper function that inserts a *bus* into the 3DCityDB. Its inputs are:
 * `db_access`: An instance of class `DBAccess`, which is the basic interface of package `dblayer` to interface a 3DCityDB
 * `bus_csv_data`: An instance of class `pandas.Series`, containing information about the bus (see below).
 * `spatial_reference_id`: Spatial reference ID for the network.
 * `network_id`: ID of the *network*, to which the *network feature* representing this bus will be added in the 3DCityDB.
 * `network_graph_id`: ID of the *network graph*, to which the *feature graph* of this bus will be added in the 3DCityDB.

This function relies of the helper function `insert_ntw_feature_2dpoint`, which adds a new newtwork feature, represented by a 2D point. In addition to the new network feature, a feature graph containing a single exterior node is created.

Function `insert_ntw_feature_2dpoint` in turn uses function `insert_ntw_feat_simple_funct_elem`, which means that the bus will be added to the 3DCityDB as a network feature of type `SimpleFunctionalElement`.

Furthermore, function `add_generic_attributes` is used to add the generic attribute *vn_kv* for this bus.

In [4]:
def write_bus_to_db(
    db_access,
    bus_csv_data,
    spatial_reference_id,
    network_id,
    network_graph_id):

    (feature_id, feature_graph_id, node_id) = insert_ntw_feature_2dpoint(
        db_access,
        insert_ntw_feat_simple_funct_elem,
        spatial_reference_id,
        network_id,
        network_graph_id,
        bus_csv_data.name,
        Point2D( bus_csv_data.x, bus_csv_data.y ),
        class_name = bus_csv_data.type
    )

    add_generic_attributes(
        db_access,
        feature_id,
        {'vn_kv': bus_csv_data.vn_kv}
    )

    return BusData(feature_id, feature_graph_id, node_id, bus_csv_data.x, bus_csv_data.y)

### A function for adding terminal elements

Next, define a similar function for adding a *terminal element* to the network. In this case, function `insert_and_link_ntw_feature_2dpoint` from package `dblayer` is used. This function does the same thing as function `insert_ntw_feature_2dpoint` and in addition connects the generated exterior node via an inter feature link to another exterior node.

Function `insert_and_link_ntw_feature_2dpoint` in turn uses function `insert_ntw_feat_term_elem`, which means that a network feature of type `TerminalElement` will be added to the 3DCityDB.

In [5]:
def write_terminal_element_to_db(
    db_access,
    name,
    terminal_type,
    connected_bus,
    spatial_reference_id,
    network_id,
    network_graph_id,
    cityobject_id = None):

    insert_and_link_ntw_feature_2dpoint(
        db_access,
        insert_ntw_feat_term_elem,
        spatial_reference_id,
        network_id,
        network_graph_id,
        name,
        Point2D( connected_bus.posx, connected_bus.posy ),
        connected_bus.node_id,
        class_name = terminal_type
    )

### A function for adding lines

Next, define a similar function for adding a *line* to the network. In this case, function `insert_and_link_ntw_feature_2dlinestring` from package `dblayer` is used. This function adds a new newtwork feature, represented by a 2D line segment (made up of 2D points). In addition to the new network feature, a feature graph containing a 2D line segment is created and the start and end points of this line segment are associated to exterior nodes and linked via an interior feature link.

Function `insert_and_link_ntw_feature_2dlinestring` in turn uses function `insert_ntw_feat_distrib_elem_cable`, which means that a network feature of type `Cable` will be added to the 3DCityDB.

In [6]:
def write_line_to_db(
    db_access,
    name,
    from_bus,
    to_bus,
    line_type,
    spatial_reference_id,
    network_id,
    network_graph_id):

    line_segment = [
        Point2D(from_bus.posx, from_bus.posy),
        Point2D(to_bus.posx, to_bus.posy)
    ]

    insert_and_link_ntw_feature_2dlinestring(
        db_access,
        insert_ntw_feat_distrib_elem_cable,
        spatial_reference_id,
        network_id,
        network_graph_id,
        name,
        line_segment,
        from_bus.node_id,
        to_bus.node_id,
        class_name = 'line',
        function = line_type
    )

### A function for adding switches

This function is again very similar to the previous one. The biggest difference is that function `insert_and_link_ntw_feature_2dlinestring` uses function `insert_ntw_feat_simple_funct_elem` to represent the switch in the 3DCityDB as an object of type `SimpleFunctionalElement`.

In [7]:
def write_switch_to_db(
    db_access,
    name,
    from_bus,
    to_bus,
    switch_type,
    spatial_reference_id,
    network_id,
    network_graph_id):

    line_segment = [
        Point2D(from_bus.posx, from_bus.posy),
        Point2D(to_bus.posx, to_bus.posy)
    ]

    insert_and_link_ntw_feature_2dlinestring(
        db_access,
        insert_ntw_feat_simple_funct_elem,
        spatial_reference_id,
        network_id,
        network_graph_id,
        name,
        line_segment,
        from_bus.node_id,
        to_bus.node_id,
        class_name = 'switch',
        function = switch_type
    )

### A function for adding transformers

Next, define a function for adding a transformer. Basically works the same as the previous two functions, but relies on function `insert_ntw_feat_simple_funct_elem` to represent a transformer as an object of type `ComplexFunctionalElement`.

In [8]:
def write_transformer_to_db(
    db_access,
    name,
    hv_bus,
    lv_bus,
    transformer_type,
    spatial_reference_id,
    network_id,
    network_graph_id):

    line_segment = [
        Point2D(hv_bus.posx, hv_bus.posy),
        Point2D(lv_bus.posx, lv_bus.posy)
    ]

    insert_and_link_ntw_feature_2dlinestring(
        db_access,
        insert_ntw_feat_complex_funct_elem,
        spatial_reference_id,
        network_id,
        network_graph_id,
        name,
        line_segment,
        hv_bus.node_id,
        lv_bus.node_id,
        class_name = 'transformer',
        function = transformer_type,
    )

### A function for adding loads
Finally, define a function that can be used to add loads to the network. To this end, it adds an object of type `ElectricalAppliances` to the database, which stores the (static) power consumption. This is done with the help of function `add_citydb_object`, which in turn uses function `insert_electrical_appliances`. Additional parameters defining the load are associated as generic parameters to this object.

Finally, a terminal element is added to the network using function `write_terminal_element_to_db` , which is linked to the electrical appliance via its city object ID.

In [9]:
def write_load_to_db(
    db_access,
    name,
    bus,
    spatial_reference_id,
    network_id,
    network_graph_id,
    p_kw,
    q_kvar,
    scaling):

    appliance_id = access.add_citydb_object(
        insert_electrical_appliances,
        name = 'electrical_appliance_' + name,
        electr_pwr = p_kw,
        electr_pwr_unit = 'kW'
    )

    add_generic_attributes(
        db_access,
        appliance_id,
        {'q_kvar': q_kvar, 'scaling': scaling}
    )

    write_terminal_element_to_db(
        db_access,
        name,
        'load',
        bus,
        spatial_reference_id,
        network_id,
        network_graph_id,
        appliance_id
    )

## Connect to database

Define connection parameters for 3DCityDB instance.

In [10]:
connect = PostgreSQLConnectionInfo(
    user='postgres',
    pwd='postgres',
    host='localhost',
    port='5432',
    dbname='testdb'
    )

Connect to database using an instance of class `DBAccess`.

In [22]:
access = DBAccess()
access.connect_to_citydb(connect)

For the purpose of this notebook yoi might want to use an empty 3DCityDB. To this end, the next two lines erase all data from an existing database. **Only execute the next two lines if you really want to erase all data from your database!**

In [12]:
access.cleanup_citydb_schema()
access.cleanup_simpkg_schema()

## Write the network to the database
Define the spatial reference ID (SRID) of the network. This should be consistent with the setup of your database.

In [13]:
srid = 25833

Insert a new *network* and an associated *network graph*. The *network* basically contains the topographical representation - the so-called *network features* - of all the network network components. The *network graph* contains the toplogical representation - of all these components the topological links between them.

In [14]:
ntw_id = access.add_citydb_object(
    insert_network,
    name = 'CIGRE_example_network'
    )

ntw_graph_id = access.add_citydb_object(
    insert_network_graph,
    name = 'CIGRE_example_network_graph',
    network_id = ntw_id
    )

Add all busses to the network. Use function `write_bus_to_db` defined above.

In [15]:
bus_data = {}

for index, bus_csv_data in busses_csv_data.iterrows():

    bus_data[bus_csv_data.name] = write_bus_to_db( 
        access,
        bus_csv_data,
        srid,
        ntw_id,
        ntw_graph_id
    )

Add all lines to the network and connect them with the busses. Use function `write_line_to_db` defined above.

In [16]:
for index, line_csv_data in lines_csv_data.iterrows():

    write_line_to_db(
        access,
        line_csv_data.name,
        bus_data[line_csv_data.from_bus],
        bus_data[line_csv_data.to_bus],
        line_csv_data.type,
        srid,
        ntw_id,
        ntw_graph_id
    )

Add all transformers to the network and connect them with the busses. Use function `write_transformer_to_db` defined above.

In [17]:
write_transformer_to_db(
    access,
    'tr-R0-R1',
    bus_data['bus-R0-MV'],
    bus_data['bus-R1-LV'],
    '0.63 MVA 20/0.4 kV',
    srid,
    ntw_id, ntw_graph_id
)

write_transformer_to_db(
    access,
    'tr-I0-I1',
    bus_data['bus-I0-MV'],
    bus_data['bus-I1-LV'],
    '0.25 MVA 20/0.4 kV',
    srid,
    ntw_id,
    ntw_graph_id
)

write_transformer_to_db(
    access,
    'tr-C0-C1',
    bus_data['bus-C0-MV'],
    bus_data['bus-C1-LV'],
    '0.4 MVA 20/0.4 kV',
    srid,
    ntw_id, ntw_graph_id
)

Add all switches to the network and connect them with the busses. Use function `write_switch_to_db` defined above.

In [18]:
write_switch_to_db(
    access,
    'switch-S1',
    bus_data['bus-0-MV'],
    bus_data['bus-R0-MV'],
    'CB',
    srid,
    ntw_id, ntw_graph_id
)

write_switch_to_db(
    access,
    'switch-S2',
    bus_data['bus-0-MV'],
    bus_data['bus-I0-MV'],
    'CB',
    srid,
    ntw_id, ntw_graph_id
)

write_switch_to_db(
    access,
    'switch-S3',
    bus_data['bus-0-MV'],
    bus_data['bus-C0-MV'],
    'CB',
    srid,
    ntw_id,
    ntw_graph_id
)

Add the external grid as terminal element. Use function `write_terminal_element_to_db` defined above.

In [19]:
write_terminal_element_to_db(
    access,
    'feeder',
    'external-grid',
    bus_data['bus-0-MV'],
    srid,
    ntw_id,
    ntw_graph_id
)

Add the loads to the network. Use function `` defined above.

In [20]:
for index, load_csv_data in loads_csv_data.iterrows():
    write_load_to_db(
        access,
        load_csv_data.name,
        bus_data[load_csv_data.bus],
        srid,
        ntw_id,
        ntw_graph_id,
        load_csv_data.p_kw,
        load_csv_data.q_kvar,
        load_csv_data.scaling
    )

Commit changes to database.

In [21]:
access.commit_citydb_session()

Done.